# Income Qualification in Latin America Using Random Forest

### By Sayorn Chin

### 04-10-2021

#### Objective
Many social programs have a hard time ensuring that the right people are given enough aid. It’s tricky when a program focuses on the poorest segment of the population. This segment of the population can’t provide the necessary income and expense records to prove that they qualify.

In Latin America, a popular method called Proxy Means Test (PMT) uses an algorithm to verify income qualification. With PMT, agencies use a model that considers a family’s observable household attributes like the material of their walls and ceiling or the assets found in their homes to
classify them and predict their level of need.

While this is an improvement, accuracy remains a problem as the region’s population grows and poverty declines. The Inter-American Development Bank (IDB) believes that new methods beyond traditional econometrics, based on a dataset of Costa Rican household characteristics, might help improve PMT’s performance.

In [249]:
# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from scipy.stats import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
%matplotlib inline

In [250]:
# Define working directory
os.chdir('/Users/schinlfc/income_qualification/data')

In [251]:
# Import datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [252]:
# Identify the output variable
for i in train.columns:
    if i not in test.columns:
        print(f" The output variable is: {i}")


 The output variable is: Target


In [253]:
# Get the first 5 rows of the data
train.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [254]:
# Understand the type of data
for i in train.columns:
    type=train[i].dtype
    if type == 'object':
        print(f"This variable is of object type: {i}")
    elif type == 'float64':
        print(f"This variable is of float type: {i}")
    elif type == 'int64':
        print(f"This variable is of integer type: {i}")
    else:
        print(f"This variable is other than object, float, or integer: {i}")

This variable is of object type: Id
This variable is of float type: v2a1
This variable is of integer type: hacdor
This variable is of integer type: rooms
This variable is of integer type: hacapo
This variable is of integer type: v14a
This variable is of integer type: refrig
This variable is of integer type: v18q
This variable is of float type: v18q1
This variable is of integer type: r4h1
This variable is of integer type: r4h2
This variable is of integer type: r4h3
This variable is of integer type: r4m1
This variable is of integer type: r4m2
This variable is of integer type: r4m3
This variable is of integer type: r4t1
This variable is of integer type: r4t2
This variable is of integer type: r4t3
This variable is of integer type: tamhog
This variable is of integer type: tamviv
This variable is of integer type: escolari
This variable is of float type: rez_esc
This variable is of integer type: hhsize
This variable is of integer type: paredblolad
This variable is of integer type: paredzocalo

In [255]:
# Drop columns.
train.drop(['Id','idhogar'],axis=1,inplace=True)
test.drop('r4t3',axis=1,inplace=True)
test.drop(['Id','idhogar'],axis=1,inplace=True)

In [256]:
# Explore object data type further
for i in train.columns:
    type=train[i].dtype
    if type == 'object':
        print(f"This variable is of object type: {i}")

This variable is of object type: dependency
This variable is of object type: edjefe
This variable is of object type: edjefa


In [257]:
train['dependency'].value_counts()

yes          2192
no           1747
.5           1497
2             730
1.5           713
.33333334     598
.66666669     487
8             378
.25           260
3             236
4             100
.75            98
.2             90
.40000001      84
1.3333334      84
2.5            77
5              24
1.25           18
3.5            18
.80000001      18
2.25           13
.71428573      12
1.75           11
1.2            11
.83333331      11
.22222222      11
.2857143        9
1.6666666       8
.60000002       8
6               7
.16666667       7
Name: dependency, dtype: int64

In [258]:
train['edjefe'].value_counts()

no     3762
6      1845
11      751
9       486
3       307
15      285
8       257
7       234
5       222
14      208
17      202
2       194
4       137
16      134
yes     123
12      113
10      111
13      103
21       43
18       19
19       14
20        7
Name: edjefe, dtype: int64

In [259]:
train['edjefa'].value_counts()

no     6230
6       947
11      399
9       237
8       217
15      188
7       179
5       176
3       152
4       136
14      120
16      113
10       96
2        84
17       76
12       72
yes      69
13       52
21        5
19        4
18        3
20        2
Name: edjefa, dtype: int64

In [260]:
def label(x):
    if x=='yes':
        return(float(1))
    elif x=='no':
        return(float(0))
    else:
        return(float(x))

In [261]:
train['dependency']=train['dependency'].apply(label)
train['edjefe']=train['edjefe'].apply(label)
train['edjefa']=train['edjefa'].apply(label)
test['dependency']=test['dependency'].apply(map)
test['edjefe']=test['edjefe'].apply(map)
test['edjefa']=test['edjefa'].apply(map)

In [262]:
# Check if there are any biases in the dataset
crosstab=pd.crosstab(train['r4t3'],train['hogar_total'])
observed_values=crosstab.values
b=scipy.stats.chi2_contingency(crosstab)
expected_values = b[3]
no_of_rows=len(crosstab.iloc[0:2,0])
no_of_columns=len(crosstab.iloc[0,0:2])
df=(no_of_rows-1)*(no_of_columns-1)
print(f"Degree of Freedom is: {df}")
chi_square=sum([(o-e)**2./e for o,e in zip(observed_values,expected_values)])
chi_square_statistic=chi_square[0]+chi_square[1]
print(f"chi-square statistic is: {chi_square_statistic}")
alpha=0.05
critical_value=chi2.ppf(q=1-alpha, df=df)
print(f"critical_value is: {critical_value}")
p_value=1-chi2.cdf(x=chi_square_statistic, df=df)
print(f"p-value is: {p_value}")
print(f"Significance level is: {alpha}")
print(f"Degree of Freedom is: {df}")
print(f"chi-square statistic is: {chi_square_statistic}")
print(f"critical_value is: {critical_value}")
print(f"p-value is: {p_value}")
if chi_square_statistic >= critical_value:
    print("Reject H0")
else:
    print("Retain H0")
if p_value <= alpha:
    print("Reject H0")
else:
    print("Retain H0")

Degree of Freedom is: 1
chi-square statistic is: 17022.072400560897
critical_value is: 3.841458820694124
p-value is: 0.0
Significance level is: 0.05
Degree of Freedom is: 1
chi-square statistic is: 17022.072400560897
critical_value is: 3.841458820694124
p-value is: 0.0
Reject H0
Reject H0


In [263]:
train.drop('r4t3',axis=1,inplace=True)

In [264]:
# Check if there is a house without a family head
pd.crosstab(train['edjefa'],train['edjefe'])

edjefe,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0
edjefa,,,,,,,,,,,,,,,,,,,,,
0.0,435,123,194,307,137,222,1845,234,257,486,...,113,103,208,285,134,202,19,14,7,43
1.0,69,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.0,84,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.0,152,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,136,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.0,176,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6.0,947,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7.0,179,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8.0,217,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


There are 435 families without a family head!

In [265]:
# Count how many null values are existing in columns
train.isna().sum()

v2a1               6860
hacdor                0
rooms                 0
hacapo                0
v14a                  0
                   ... 
SQBovercrowding       0
SQBdependency         0
SQBmeaned             5
agesq                 0
Target                0
Length: 140, dtype: int64

In [266]:
float_cols=[]
for i in train.columns:
    type=train[i].dtype
    if type == 'float64':
        float_cols.append(i)
print(float_cols)

['v2a1', 'v18q1', 'rez_esc', 'dependency', 'edjefe', 'edjefa', 'meaneduc', 'overcrowding', 'SQBovercrowding', 'SQBdependency', 'SQBmeaned']


In [267]:
train[float_cols].isna().sum()

v2a1               6860
v18q1              7342
rez_esc            7928
dependency            0
edjefe                0
edjefa                0
meaneduc              5
overcrowding          0
SQBovercrowding       0
SQBdependency         0
SQBmeaned             5
dtype: int64

In [268]:
train['v2a1'].fillna(0,inplace=True)
train['v18q1'].fillna(0,inplace=True)
test['v2a1'].fillna(0,inplace=True)
test['v18q1'].fillna(0,inplace=True)

In [269]:
train.drop(['tipovivi3', 'v18q','rez_esc','elimbasu5'],axis=1,inplace=True)
test.drop(['tipovivi3', 'v18q','rez_esc','elimbasu5'],axis=1,inplace=True)

In [270]:
train['meaneduc'].fillna(np.mean(train['meaneduc']),inplace=True)
train['SQBmeaned'].fillna(np.mean(train['SQBmeaned']),inplace=True)
test['meaneduc'].fillna(np.mean(test['meaneduc']),inplace=True)
test['SQBmeaned'].fillna(np.mean(test['SQBmeaned']),inplace=True)
print(f"train: {train.isna().sum()} and test: {test.isna().sum()}")

train: v2a1               0
hacdor             0
rooms              0
hacapo             0
v14a               0
                  ..
SQBovercrowding    0
SQBdependency      0
SQBmeaned          0
agesq              0
Target             0
Length: 136, dtype: int64 and test: v2a1               0
hacdor             0
rooms              0
hacapo             0
v14a               0
                  ..
SQBhogar_nin       0
SQBovercrowding    0
SQBdependency      0
SQBmeaned          0
agesq              0
Length: 135, dtype: int64


In [271]:
# Set the poverty level of the members and the head of the house same in a family
poverty_level = train[train['v2a1'] != 0]

In [272]:
poverty_threshold = poverty_level.groupby('area1')['v2a1'].apply(np.median)

In [273]:
poverty_threshold

area1
0     80000.0
1    140000.0
Name: v2a1, dtype: float64

In [274]:
def poverty_label(x):
    if x < 8000:
        return('Below poverty level')
    elif x > 140000:
        return('Above poverty level')
    elif x < 140000:
        return('Below poverty level: Urban; Above poverty level: Rural')

In [275]:
pd.crosstab(poverty_level['v2a1'].apply(poverty_label), poverty_level['area1'])

area1,0,1
v2a1,,
Above poverty level,139,1103
Below poverty level: Urban; Above poverty level: Rural,306,1081


#### Predict the accuracy using random forest classifier

In [276]:
x = train.drop('Target',axis=1)
y = train.Target

In [277]:
x_cols = x.columns

In [278]:
SS = StandardScaler()
X = SS.fit_transform(x)
X  = pd.DataFrame(X, columns = x_cols)

In [279]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.30, stratify= y, random_state=100)

In [280]:
rfc=RandomForestClassifier(random_state=0)
parameters={'n_estimators':[10,50,100,300],'max_depth':[3,5,10,15]}
grid=zip([rfc],[parameters])

best_=None

for i, j in grid:
    a=GridSearchCV(i,param_grid=j,cv=3,n_jobs=1)
    a.fit(X_train,Y_train)
    if best_ is None:
        best_=a
    elif a.best_score_>best_.best_score_:
        best_=a
        
        
print ("Best CV Score",best_.best_score_)
print ("Model Parameters",best_.best_params_)
print("Best Estimator",best_.best_estimator_)

Best CV Score 0.8437725028349633
Model Parameters {'max_depth': 15, 'n_estimators': 300}
Best Estimator RandomForestClassifier(max_depth=15, n_estimators=300, random_state=0)


In [281]:
RFC=best_.best_estimator_
Model=RFC.fit(X_train,Y_train)
pred=Model.predict(X_test)

In [282]:
print('Model Score of train data : {}'.format(Model.score(X_train,Y_train)))
print('Model Score of test data : {}'.format(Model.score(X_test,Y_test)))

Model Score of train data : 0.9802661085364031
Model Score of test data : 0.8824965132496513


In [283]:
Important_features=pd.DataFrame(Model.feature_importances_,x_cols,columns=['feature_importance'])

In [284]:
Top70Features=Important_features.sort_values(by='feature_importance',ascending=False).head(70).index

In [285]:
for i in Top70Features:
    if i not in x_cols:
        print(i)

In [286]:
X_data_Top70 = x[Top70Features]

In [287]:
X_train,X_test,Y_train,Y_test=train_test_split(X_data_Top70, y, test_size=0.30, stratify=y,random_state=100)

In [288]:
model = RFC.fit(X_train,Y_train)
pred = model.predict(X_test)

In [289]:
f1_score(Y_test,pred,average='weighted')

0.9066708095356174

In [290]:
accuracy_score(Y_test,pred)

0.9100418410041841

In [291]:
# Now using the given test dataset to make prediction
test_data=test[Top70Features]

In [292]:
test_dataset = SS.fit_transform(test_data)
X =pd.DataFrame(test_dataset)

In [293]:
test_prediction=model.predict(test_data)

In [294]:
print(test_prediction)

[4 4 4 ... 4 4 4]


#### Conclusion
The accuracy score using Random Forest is 91%